# Laboratorio #3
---
Nancy Mazariegos 22513

Santiago Pereira 22318

---

In [ ]:
# Importaciones y parámetros definidos
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

N = 10000 # Poblacion
I0 = 10 # Infectados iniciales
R0 = 0
S0 = N - I0 - R0
beta = 0.35
gamma = 0.1
days = 160
t = np.linspace(0, days, days)